In [5]:
%pip install -U layoutparser


You should consider upgrading via the 'c:\Users\pwayk\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
%pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2' 

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2'"
Hint: = is not a valid operator. Did you mean == ?
You should consider upgrading via the 'c:\Users\pwayk\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [7]:
%pip install tesseract-ocr

  Using cached tesseract-ocr-0.0.1.tar.gz (33 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for tesseract-ocr
Failed to build tesseract-ocr
  Running setup.py install for tesseract-ocr: started
  Running setup.py install for tesseract-ocr: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [16 lines of output]
      c:\Users\pwayk\AppData\Local\Programs\Python\Python39\lib\site-packages\setuptools\dist.py:697: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
        warnings.warn(
      running bdist_wheel
      running build
      running build_py
      file tesseract_ocr.py (for module tesseract_ocr) not found
      file tesseract_ocr.py (for module tesseract_ocr) not found
      running build_ext
      building 'tesseract_ocr' extension
      creating build
      creating build\temp.win-amd64-3.9
      creating build\temp.win-amd64-3.9\Release
      C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Tools\MSVC\14.31.31103\bin\HostX86\x64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -Ic:\Users\pwayk\AppData\Local\Programs\Python\Python39\

In [8]:
%pip install layoutparser[ocr]     

You should consider upgrading via the 'c:\Users\pwayk\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [9]:
!git clone https://github.com/Layout-Parser/layout-parser.git

Cloning into 'layout-parser'...
error: RPC failed; curl 56 OpenSSL SSL_read: Connection was reset, errno 10054
error: 3159 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [10]:
import cv2
# from google.colab.patches import cv2_imshow

image = cv2.imread("images\out0.jpg")
image = image[..., ::-1] 
cv2.imshow("image",image)
cv2.waitKey(0)
#/content/Injection (1).jpg

-1

In [24]:
import layoutparser as lp
ocr_agent = lp.TesseractAgent(languages='eng') 

In [25]:

model = lp.Detectron2LayoutModel('lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config',extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.65],
                                 label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})
layout = model.detect(image) # You need to load the image somewhere else, e.g., image = cv2.imread(...)
lp.draw_box(image, layout,)

AttributeError: module layoutparser has no attribute Detectron2LayoutModel

In [ ]:
len(layout)

17

In [ ]:
text_blocks = lp.Layout([b for b in layout if b.type=="Text"])
text_blocks

Layout(_blocks=[], page_data={})

In [ ]:
figure_blocks = lp.Layout([b for b in layout if b.type=='Figure'])

In [ ]:
figure_blocks

Layout(_blocks=[TextBlock(block=Rectangle(x_1=179.95457458496094, y_1=0.0, x_2=631.433349609375, y_2=480.0), text=None, id=None, type=Figure, parent=None, next=None, score=0.7257561087608337)], page_data={})

In [ ]:

text_blocks = lp.Layout([b for b in text_blocks \
                   if not any(b.is_in(b_fig) for b_fig in figure_blocks)])
h, w = image.shape[:2]

left_interval = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(image)

left_blocks = text_blocks.filter_by(left_interval, center=True)
left_blocks.sort(key = lambda b:b.coordinates[1])

right_blocks = [b for b in text_blocks if b not in left_blocks]
right_blocks.sort(key = lambda b:b.coordinates[1])

# And finally combine the two list and add the index
# according to the order
text_blocks = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks + right_blocks)])

lp.draw_box(image, text_blocks,
            box_width=3, 
            show_element_id=True)
for block in text_blocks:
    segment_image = (block
                       .pad(left=5, right=5, top=5, bottom=5)
                       .crop_image(image))
        # add padding in each image segment can help
        # improve robustness 
        
    text = ocr_agent.detect(segment_image)
    block.set(text=text, inplace=True)
text_list = []
for txt in text_blocks.get_texts():
    print(txt, end='\n---\n')
    text_list.append(txt)


Injection molding is used for manufacturing a wide variety of parts, frorr
small components like AAA battery boxes to large components like truck
body panels. Once a component is designed, a mold is made and preci-
sion machined to form the features of the desired part. The injection mold-
ing takes place when a thermoplastic or thermoset plastic material is fec
into a heated barrel, mixed, and forced into the metal mold cavity where i
cools and hardens before being removed.

---
Mold and die are used interchangeably to describe the tooling applied to
produce plastic parts. They are typically constructed from pre-hardened
steel, hardened steel, aluminum, and/or beryllium-copper alloy. Of these
materials, hardened steel molds are the most expensive to make, but offer
the user a long lifespan, which offsets the cost per part by spreading it over
a larger quantity. For low volumes or large components, pre-hardened steel
molds provide a less wear-resistant and less expensive option.

---

In [ ]:
text_list[0]

'Injection molding is used for manufacturing a wide variety of parts, frorr\nsmall components like AAA battery boxes to large components like truck\nbody panels. Once a component is designed, a mold is made and preci-\nsion machined to form the features of the desired part. The injection mold-\ning takes place when a thermoplastic or thermoset plastic material is fec\ninto a heated barrel, mixed, and forced into the metal mold cavity where i\ncools and hardens before being removed.\n\x0c'

In [ ]:
len(text_list)

4

In [ ]:
text_list

['Injection molding is used for manufacturing a wide variety of parts, frorr\nsmall components like AAA battery boxes to large components like truck\nbody panels. Once a component is designed, a mold is made and preci-\nsion machined to form the features of the desired part. The injection mold-\ning takes place when a thermoplastic or thermoset plastic material is fec\ninto a heated barrel, mixed, and forced into the metal mold cavity where i\ncools and hardens before being removed.\n\x0c',
 'Mold and die are used interchangeably to describe the tooling applied to\nproduce plastic parts. They are typically constructed from pre-hardened\nsteel, hardened steel, aluminum, and/or beryllium-copper alloy. Of these\nmaterials, hardened steel molds are the most expensive to make, but offer\nthe user a long lifespan, which offsets the cost per part by spreading it over\na larger quantity. For low volumes or large components, pre-hardened steel\nmolds provide a less wear-resistant and less expen

###Titles Extraction into List


In [ ]:
title_blocks = lp.Layout([b for b in layout if b.type=='Title'])

h_1, w_1 = image.shape[:2]

left_interval_1 = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(image)

left_blocks_1 = title_blocks.filter_by(left_interval_1, center=True)
left_blocks_1.sort(key = lambda b:b.coordinates[1], inplace=True)

right_blocks_1 = [b for b in title_blocks if b not in left_blocks_1]
right_blocks_1.sort(key = lambda b:b.coordinates[1])
# And finally combine the two list and add the index
# according to the order
title_blocks = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks_1 + right_blocks_1)])

lp.draw_box(image, title_blocks,
            box_width=3,
            show_element_id=True)
for block in title_blocks:
    segment_image = (block
                       .pad(left=5, right=5, top=5, bottom=5)
                       .crop_image(image))
        # add padding in each image segment can help
        # improve robustness

    text_1 = ocr_agent.detect(segment_image)
    block.set(text=text_1, inplace=True)

title_list = []
for txt in title_blocks.get_texts():
    print(txt, end='\n---\n')
    title_list.append(txt)

print(title_list)

TOOLING

---
INJECTION MOLDING

---
NJECTION MOLDED PARTS

---
['TOOLING\n\x0c', 'INJECTION MOLDING\n\x0c', 'NJECTION MOLDED PARTS\n\x0c']


In [ ]:
title_list

['RIB INTERSECTIONS\n\x0c',
 'RIB GUILDELINES\n\x0c',
 'RIB/LOAD AFFECT ON STIFFNESS\n\x0c',
 'DRAFT AND TEXTURE\n\x0c']

In [ ]:
title_blocks[0]

TextBlock(block=Rectangle(x_1=76.74003601074219, y_1=290.5187683105469, x_2=303.3755187988281, y_2=327.1947326660156), text=RIB INTERSECTIONS

, id=0, type=Title, parent=None, next=None, score=0.9743063449859619)

In [ ]:
title_blocks[1]

TextBlock(block=Rectangle(x_1=75.00328063964844, y_1=820.4400024414062, x_2=270.156982421875, y_2=857.1689453125), text=RIB GUILDELINES

, id=1, type=Title, parent=None, next=None, score=0.9905503988265991)

In [ ]:
title_blocks[2]

###Extraction of Title and Paragraph 

In [ ]:
len(layout)

17

In [ ]:
para_title_blocks = lp.Layout([b for b in layout])

h_2, w_2 = image.shape[:2]

left_interval_2 = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(image)

left_blocks_2 = para_title_blocks.filter_by(left_interval_2, center=True)
left_blocks_2.sort(key = lambda b:b.coordinates[1], inplace=True)

right_blocks_2 = [b for b in para_title_blocks if b not in left_blocks_2]
right_blocks_2.sort(key = lambda b:b.coordinates[1])
# And finally combine the two list and add the index
# according to the order
para_title_blocks = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks_2 + right_blocks_2)])

lp.draw_box(image, para_title_blocks,
            box_width=3,
            show_element_id=True)
for block in para_title_blocks:
    segment_image = (block
                       .pad(left=5, right=5, top=5, bottom=5)
                       .crop_image(image))
        # add padding in each image segment can help
        # improve robustness

    text_2 = ocr_agent.detect(segment_image)
    block.set(text=text_2, inplace=True)

para_title_list = []
for txt in para_title_blocks.get_texts():
    print(txt, end='\n---\n')
    para_title_list.append(txt)

print(para_title_list)

INJECT]

 

---
TOOLING

---
INJECTION MOLDING

---
NJECTION MOLDED PARTS

---
Injection molding is used for manufacturing a wide variety of parts, frorr
small components like AAA battery boxes to large components like truck
body panels. Once a component is designed, a mold is made and preci-
sion machined to form the features of the desired part. The injection mold-
ing takes place when a thermoplastic or thermoset plastic material is fec
into a heated barrel, mixed, and forced into the metal mold cavity where i
cools and hardens before being removed.

---
Mold and die are used interchangeably to describe the tooling applied to
produce plastic parts. They are typically constructed from pre-hardened
steel, hardened steel, aluminum, and/or beryllium-copper alloy. Of these
materials, hardened steel molds are the most expensive to make, but offer
the user a long lifespan, which offsets the cost per part by spreading it over
a larger quantity. For low volumes or large components, pre-

In [ ]:
len(para_title_list)

17

In [ ]:
para_title_list[2]

'Core at Rib\nIntersection\nMinimizes Sink\n\n    \n\n\\)\n\x0c'

In [ ]:
type(title_blocks)

#### KeyBERT

In [ ]:
#########################

In [ ]:
#text_blocks.get_texts()
print(text_list)

['A rib is oriented in such a way as to provide maximum\nbending stiffness to the part. By paying attention to part\ngeometry, designers must be conscious of the orienta-\ntion of the rib to the bending load or there will be no\nincrease in stiffness.\n\x0c', 'The height of a rib should be limited to less than three\ntimes its thickness. It is better to use multiple ribs to\nincrease bending stiffness than to use one very tall rib.\n\x0c', 'Because the thickness of the material will be greater at\nthe rib intersections, coring or another means of ma-\nterial removal should be employed to avoid excessive\nsinking from occurring on the opposite side.\n\x0c', 'Figure 10: Coring at rib intersections\n\x0c', 'Figure 11: Design guidelines for ribs\n\x0c', 'Figure 12: Rib/ load orientation affects part stiffness;\nDraft angles for ribs should be a minimum of 0.25 to 0.5\ndegree of draft per side\n\x0c', 'Mold drafts facilitate part removal from the mold. The\ndraft must be in an offset angle 

In [ ]:
#################################

In [ ]:
text_list

['A rib is oriented in such a way as to provide maximum\nbending stiffness to the part. By paying attention to part\ngeometry, designers must be conscious of the orienta-\ntion of the rib to the bending load or there will be no\nincrease in stiffness.\n\x0c',
 'The height of a rib should be limited to less than three\ntimes its thickness. It is better to use multiple ribs to\nincrease bending stiffness than to use one very tall rib.\n\x0c',
 'Because the thickness of the material will be greater at\nthe rib intersections, coring or another means of ma-\nterial removal should be employed to avoid excessive\nsinking from occurring on the opposite side.\n\x0c',
 'Figure 10: Coring at rib intersections\n\x0c',
 'Figure 11: Design guidelines for ribs\n\x0c',
 'Figure 12: Rib/ load orientation affects part stiffness;\nDraft angles for ribs should be a minimum of 0.25 to 0.5\ndegree of draft per side\n\x0c',
 'Mold drafts facilitate part removal from the mold. The\ndraft must be in an offset 

In [ ]:
para_title_list

['RIB INTERSECTIONS\n\x0c',
 'Because the thickness of the material will be greater at\nthe rib intersections, coring or another means of ma-\nterial removal should be employed to avoid excessive\nsinking from occurring on the opposite side.\n\x0c',
 'Core at Rib\nIntersection\nMinimizes Sink\n\n    \n\n\\)\n\x0c',
 'Figure 10: Coring at rib intersections\n\x0c',
 'RIB GUILDELINES\n\x0c',
 'The height of a rib should be limited to less than three\ntimes its thickness. It is better to use multiple ribs to\nincrease bending stiffness than to use one very tall rib.\n\x0c',
 '4\n\n \n\n \n\n \n\n— H<(3xT)\n\n \n\n \n\nS$ 2(2xT)\n\n \n\n[\nCt\n\n40-60% of T\n\x0c',
 'Figure 11: Design guidelines for ribs\n\x0c',
 'RIB/LOAD AFFECT ON STIFFNESS\n\x0c',
 'A rib is oriented in such a way as to provide maximum\nbending stiffness to the part. By paying attention to part\ngeometry, designers must be conscious of the orienta-\ntion of the rib to the bending load or there will be no\nincrease in sti

In [ ]:
para_title_list

NameError: ignored

In [ ]:
for i in range(n):
  print('ok')

ok
ok
ok
ok


In [ ]:
i

1

In [ ]:
dic_1

{'DRAFT AND TEXTURE\n\x0c': [],
 'RIB GUILDELINES\n\x0c': [],
 'RIB INTERSECTIONS\n\x0c': ['RIB INTERSECTIONS\n\x0c',
  'Because the thickness of the material will be greater at\nthe rib intersections, coring or another means of ma-\nterial removal should be employed to avoid excessive\nsinking from occurring on the opposite side.\n\x0c',
  'Core at Rib\nIntersection\nMinimizes Sink\n\n    \n\n\\)\n\x0c',
  'Figure 10: Coring at rib intersections\n\x0c',
  'RIB GUILDELINES\n\x0c',
  'The height of a rib should be limited to less than three\ntimes its thickness. It is better to use multiple ribs to\nincrease bending stiffness than to use one very tall rib.\n\x0c',
  '4\n\n \n\n \n\n \n\n— H<(3xT)\n\n \n\n \n\nS$ 2(2xT)\n\n \n\n[\nCt\n\n40-60% of T\n\x0c',
  'Figure 11: Design guidelines for ribs\n\x0c',
  'RIB/LOAD AFFECT ON STIFFNESS\n\x0c',
  'A rib is oriented in such a way as to provide maximum\nbending stiffness to the part. By paying attention to part\ngeometry, designers must be 

In [ ]:
dic = {}
for j in title_list:
  dic['%s'%j]=[]


 

In [ ]:
dic

{'DRAFT AND TEXTURE\n\x0c': [],
 'RIB GUILDELINES\n\x0c': [],
 'RIB INTERSECTIONS\n\x0c': [],
 'RIB/LOAD AFFECT ON STIFFNESS\n\x0c': []}

In [ ]:
list(dic.keys())[0]

'RIB INTERSECTIONS\n\x0c'

In [ ]:
dic[list(dic.keys())[0]]

[]

In [ ]:
type(dic[list(dic.keys())[0]])

list

In [ ]:
type(dic['DRAFT AND TEXTURE\n\x0c'])

list

In [ ]:
dic[0]

KeyError: ignored

In [ ]:
for j in range(k):
    globals()['list_' + str(j)] 

KeyError: ignored

In [ ]:
dic

{'DRAFT AND TEXTURE\n\x0c': [],
 'RIB GUILDELINES\n\x0c': [],
 'RIB INTERSECTIONS\n\x0c': [],
 'RIB/LOAD AFFECT ON STIFFNESS\n\x0c': []}

In [ ]:
i = 1
dic['%s'%title_list(i)]


TypeError: ignored

In [ ]:
dic['DRAFT AND TEXTURE\n\x0c']

[]

In [ ]:
dic['lst_DRAFT AND TEXTURE\n\x0c']

[]

In [ ]:
a = title_list[0]
b = title_list[1]

if a in para_title_list:
    result_1 = para_title_list.index(a)
    print(f"The {a} has an index of {result_1}.")
else:
    print(f"{a} doesn't exist in the list.")

if b in para_title_list:
    result_2 = para_title_list.index(b)
    print(f"The {b} has an index of {result_2}.")
else:
    print(f"{b} doesn't exist in the list.")


l_1_1=[]
for i_final in para_title_list:
  if result_1 < result_2:
    print(para_title_list[result_1])
    l_1_1.append(para_title_list[result_1])
    result_1 = result_1+1
  else:
    ('done')
print(l_1_1)


The TOOLING
 has an index of 1.
The INJECTION MOLDING
 has an index of 2.
TOOLING

['TOOLING\n\x0c']


In [ ]:
text_list

['Injection molding is used for manufacturing a wide variety of parts, frorr\nsmall components like AAA battery boxes to large components like truck\nbody panels. Once a component is designed, a mold is made and preci-\nsion machined to form the features of the desired part. The injection mold-\ning takes place when a thermoplastic or thermoset plastic material is fec\ninto a heated barrel, mixed, and forced into the metal mold cavity where i\ncools and hardens before being removed.\n\x0c',
 'Mold and die are used interchangeably to describe the tooling applied to\nproduce plastic parts. They are typically constructed from pre-hardened\nsteel, hardened steel, aluminum, and/or beryllium-copper alloy. Of these\nmaterials, hardened steel molds are the most expensive to make, but offer\nthe user a long lifespan, which offsets the cost per part by spreading it over\na larger quantity. For low volumes or large components, pre-hardened steel\nmolds provide a less wear-resistant and less expen

In [ ]:
title_list

['TOOLING\n\x0c', 'INJECTION MOLDING\n\x0c', 'NJECTION MOLDED PARTS\n\x0c']

In [ ]:
a

In [ ]:
para_title_list

['INJECT]\n\n \n\x0c',
 'TOOLING\n\x0c',
 'INJECTION MOLDING\n\x0c',
 'NJECTION MOLDED PARTS\n\x0c',
 'Injection molding is used for manufacturing a wide variety of parts, frorr\nsmall components like AAA battery boxes to large components like truck\nbody panels. Once a component is designed, a mold is made and preci-\nsion machined to form the features of the desired part. The injection mold-\ning takes place when a thermoplastic or thermoset plastic material is fec\ninto a heated barrel, mixed, and forced into the metal mold cavity where i\ncools and hardens before being removed.\n\x0c',
 'Mold and die are used interchangeably to describe the tooling applied to\nproduce plastic parts. They are typically constructed from pre-hardened\nsteel, hardened steel, aluminum, and/or beryllium-copper alloy. Of these\nmaterials, hardened steel molds are the most expensive to make, but offer\nthe user a long lifespan, which offsets the cost per part by spreading it over\na larger quantity. For lo

RIB INTERSECTIONS

Because the thickness of the material will be greater at
the rib intersections, coring or another means of ma-
terial removal should be employed to avoid excessive
sinking from occurring on the opposite side.

Core at Rib
Intersection
Minimizes Sink

    

\)

Figure 10: Coring at rib intersections



In [ ]:
len(l_1_1)

4

In [ ]:
print(l_1_1)

['INJECTION MOLDING MATERIALS\n\x0c', '‘Materials Selection: Many types of thermoplastic ma-\nterials are available. Selection depends on the specific\napplication. The chart below shows some of the most\n‘common materials being used.\n\x0c', ' \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\nPolyphenylene Sulfide\nNylons Pea\nPolycarbonates Polyehter Sutfone\n= Polyetheretherketone\nPEEK\nAcrylics Fluoropolymers\nE aoa\nPolyphenylene Oxide\nPoiyethylenes peo\n‘Acrylonitrile Butadiene Polyurethanes\nStyrene PUR,\n‘Thermoplastic Polyphthalamide\n\n \n\n \n\n \n\x0c', 'WALL SECTION\nCONSIDERATIONS\n\x0c']


In [ ]:
for i_final in (para_title_list[i_1],para_title_list[i_2]):
  print(i_final)

NameError: ignored

In [ ]:
#########################################################


In [ ]:
c = title_list[1]
d = title_list[2]

if a in para_title_list:
    result_3 = para_title_list.index(c)
    print(f"The {c} has an index of {result_3}.")
else:
    print(f"{c} doesn't exist in the list.")

if d in para_title_list:
    result_4 = para_title_list.index(d)
    print(f"The {d} has an index of {result_4}.")
else:
    print(f"{d} doesn't exist in the list.")


l_1_2=[]
for i_final in para_title_list:
  if result_3 < result_4:
    print(para_title_list[result_3])
    l_1_2.append(para_title_list[result_3])
    result_3 = result_3+1
  else:
    ('done')


The INJECTION MOLDING
 has an index of 2.
The NJECTION MOLDED PARTS
 has an index of 3.
INJECTION MOLDING



INJECTION MOLDING



In [ ]:
print(l_1_2)

['RIB GUILDELINES\n\x0c', 'The height of a rib should be limited to less than three\ntimes its thickness. It is better to use multiple ribs to\nincrease bending stiffness than to use one very tall rib.\n\x0c', '4\n\n \n\n \n\n \n\n— H<(3xT)\n\n \n\n \n\nS$ 2(2xT)\n\n \n\n[\nCt\n\n40-60% of T\n\x0c', 'Figure 11: Design guidelines for ribs\n\x0c']


In [ ]:
title_list

['RIB INTERSECTIONS\n\x0c',
 'RIB GUILDELINES\n\x0c',
 'RIB/LOAD AFFECT ON STIFFNESS\n\x0c',
 'DRAFT AND TEXTURE\n\x0c']

In [ ]:
######################################################

In [ ]:
c = title_list[1]
d = title_list[2]

if a in para_title_list:
    result_3 = para_title_list.index(c)
    print(f"The {c} has an index of {result_3}.")
else:
    print(f"{c} doesn't exist in the list.")

if d in para_title_list:
    result_4 = para_title_list.index(d)
    print(f"The {d} has an index of {result_4}.")
else:
    print(f"{d} doesn't exist in the list.")

###################


l_1_2=[]
for i_final in para_title_list:
  if result_3 < result_4:
    print(para_title_list[result_3])
    l_1_2.append(para_title_list[result_3])
    result_3 = result_3+1
  else:
    ('done')


The RIB GUILDELINES
 has an index of 4.
The RIB/LOAD AFFECT ON STIFFNESS
 has an index of 8.
RIB GUILDELINES

The height of a rib should be limited to less than three
times its thickness. It is better to use multiple ribs to
increase bending stiffness than to use one very tall rib.

4

 

 

 

— H<(3xT)

 

 

S$ 2(2xT)

 

[
Ct

40-60% of T

Figure 11: Design guidelines for ribs



In [ ]:
d=1

In [ ]:
para_title_list[d]